# Нормализация базы данных до 3НФ

## Исходные данные

- **`transactions.csv`** - данные о транзакциях, продуктах и повторяющиеся атрибуты продуктов
- **`customers.csv`** - данные о клиентах

## Процесс нормализации

### 1. Первая нормальная форма (1НФ)

**Требования 1НФ:**
- Все значения атомарны
- Нет повторяющихся групп данных

**Вывод:** Исходные данные соответствовали 1НФ

### 2. Вторая нормальная форма (2НФ)

**Требования 2НФ:**
- Соответствие 1НФ
- Все неключевые атрибуты зависят от всего первичного ключа
- Нет частичных зависимостей

**Проблемы в исходных данных:**

Одни данные зависели от номера транзакции, а другие - только от товара.
Информация о самой транзакции (дата, статус, цена продажи) - разная для каждой строки
Информация о товаре (бренд, категория, себестоимость) - одинаковая для одного и того же товара, независимо от того, в какой транзакции он участвует

**Решение для приведения к 2НФ:**
- Выделена таблица `products` с первичным ключом `product_id`
- В таблице `transactions` остались только ссылки на продукты через внешний ключ

### 3. Третья нормальная форма (3НФ)

**Выявленные транзитивные зависимости:**

#### В таблице `customers`:
- **`postcode → state → country`** - строгая транзитивная зависимость
- **Повторяющиеся значения `job_industry_category`** - потенциальная зависимость

#### В таблице `products`:
- **Повторяющиеся значения `brand`** - потенциальная зависимость  
- **`product_line → product_class`** - потенциальная транзитивная зависимость

## Таблицы после нормализации

| Группа | Таблицы |
|--------|---------|
| **Справочники** | `countries`, `states`, `postcodes`, `industry_categories`, `brands`, `product_categories` |
| **Основные** | `customers`, `products`, `transactions` |

# SQL-скрипты создания таблиц

## Справочные таблицы

### Таблица countries
```sql
CREATE TABLE countries (
    country_id INTEGER PRIMARY KEY,
    country_name VARCHAR(50) UNIQUE NOT NULL
);
```

### Таблица states
```sql
CREATE TABLE states (
    state_id INTEGER PRIMARY KEY,
    state_name VARCHAR(50) NOT NULL,
    country_id INTEGER NOT NULL REFERENCES countries(country_id)
);
```

### Таблица postcodes
```sql
CREATE TABLE postcodes (
    postcode VARCHAR(20) PRIMARY KEY,
    state_id INTEGER NOT NULL REFERENCES states(state_id)
);
```

### Таблица industry_categories
```sql
CREATE TABLE industry_categories (
    industry_id INTEGER PRIMARY KEY,
    industry_name VARCHAR(50) UNIQUE NOT NULL
);
```

### Таблица brands
```sql
CREATE TABLE brands (
    brand_id INTEGER PRIMARY KEY,
    brand_name VARCHAR(100) UNIQUE NOT NULL
);
```

### Таблица product_categories
```sql
CREATE TABLE product_categories (
    category_id INTEGER PRIMARY KEY,
    product_line VARCHAR(100) NOT NULL,
    product_class VARCHAR(50) NOT NULL
);
```

## Основные таблицы

### Таблица customers
```sql
CREATE TABLE customers (
    customer_id INTEGER PRIMARY KEY,
    first_name VARCHAR(50) NOT NULL,
    last_name VARCHAR(50) NOT NULL,
    gender VARCHAR(10),
    date_of_birth DATE,
    job_title VARCHAR(100),
    job_industry_category_id INTEGER REFERENCES industry_categories(industry_id),
    wealth_segment VARCHAR(50) NOT NULL,
    deceased_indicator VARCHAR(10) NOT NULL,
    owns_car BOOLEAN NOT NULL,
    address TEXT,
    postcode VARCHAR(20) REFERENCES postcodes(postcode),
    property_valuation INTEGER
);
```

### Таблица products
```sql
CREATE TABLE products (
    product_id INTEGER PRIMARY KEY,
    brand_id INTEGER NOT NULL REFERENCES brands(brand_id),
    category_id INTEGER NOT NULL REFERENCES product_categories(category_id),
    product_size VARCHAR(50) NOT NULL,
    standard_cost DECIMAL(10,2) NOT NULL
);
```

### Таблица transactions
```sql
CREATE TABLE transactions (
    transaction_id INTEGER PRIMARY KEY,
    product_id INTEGER NOT NULL REFERENCES products(product_id),
    customer_id INTEGER NOT NULL REFERENCES customers(customer_id),
    transaction_date DATE NOT NULL,
    online_order BOOLEAN NOT NULL,
    order_status VARCHAR(50) NOT NULL,
    list_price DECIMAL(10,2) NOT NULL
);
```